In [1]:
import torch
import random
import numpy as np
import os
from tqdm import tqdm, trange
# torch.cuda.empty_cache()
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_pretrained_bert.optimization import BertAdam

In [2]:
from run_classifier import ColaProcessor, MrpcProcessor, logger, convert_examples_to_features,\
    set_optimizer_params_grad, copy_optimizer_params_to_model, accuracy, p_r_f1, tp_pcount_gcount

In [3]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    n_gpu = torch.cuda.device_count()
    
    print('There are %d GPU(s) available.' % n_gpu)

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1050 Ti


In [4]:
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from pytorch_pretrained_bert.modeling import BertForSequenceClassification


05/22/2020 15:44:32 - INFO - transformers.file_utils -   PyTorch version 1.4.0 available.
05/22/2020 15:44:35 - INFO - transformers.file_utils -   TensorFlow version 2.1.0 available.


In [5]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [6]:
# def train_and_test(data_dir, bert_model="bert-base-uncased", task_name=None,
#                    output_dir=None, max_seq_length=128, do_train=False, do_eval=False, do_lower_case=False,
#                    train_batch_size=32, eval_batch_size=8, learning_rate=5e-5, num_train_epochs=3,
#                    warmup_proportion=0.1,no_cuda=False, local_rank=-1, seed=42, gradient_accumulation_steps=1,
#                    optimize_on_cpu=False, fp16=False, loss_scale=128, saved_model=""):

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

05/22/2020 15:44:36 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\arsen\.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [8]:
# Prepare model 
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels = 2)
model.to(device)

# model = BertModel.from_pretrained('bert-base-uncased')

05/21/2020 12:42:52 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\arsen\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/21/2020 12:42:52 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file C:\Users\arsen\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\arsen\AppData\Local\Temp\tmpd0kymw7h
05/21/2020 12:42:57 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_lay

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [9]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
    
print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [14]:
data_dir = "D:/Projects/Stance/Dataset/BertForOppositeClassification/"
# data_dir = "D:/Jupyter/data/dataset/perspective_stances/"
data_dir_output = "D:/Projects/Stance/Models/dataExpantion/"
output_dir=data_dir_output
max_seq_length=32
max_grad_norm = 1.0
num_training_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_training_steps)  # 0.1
# warmup_proportion = 0.1
train_batch_size=32
eval_batch_size=8
learning_rate=5e-5
num_train_epochs=3
local_rank=-1
seed=42
gradient_accumulation_steps=1
loss_scale=128
train_batch_size = int(train_batch_size / gradient_accumulation_steps)

processors = {
        "mrpc": MrpcProcessor,
    }

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
    
os.makedirs(output_dir, exist_ok=True)
processor = processors['mrpc']()
label_list = processor.get_labels()

train_examples = processor.get_train_examples(data_dir)
num_train_steps = int(
    len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

##preprare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]
t_total = num_train_steps
optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)
# optimizer = AdamW(optimizer_grouped_parameters,
#                   lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8, # args.adam_epsilon  - default is 1e-8.
#                   correct_bias=False
#                 )

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)  # PyTorch scheduler

05/21/2020 13:28:59 - INFO - run_classifier -   LOOKING AT D:/Projects/Stance/Dataset/BertForOppositeClassification/train.tsv


In [15]:
global_step = 0
train_features = convert_examples_to_features(train_examples, label_list, max_seq_length, tokenizer)
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

05/21/2020 13:29:01 - INFO - run_classifier -   *** Example ***
05/21/2020 13:29:01 - INFO - run_classifier -   guid: train-1
05/21/2020 13:29:01 - INFO - run_classifier -   tokens: [CLS] male infant ci ##rc ##um ##cision is tan ##tam ##ount to child abuse [SEP] parents have the right to use their best judgment , in the light of medical advice [SEP]
05/21/2020 13:29:01 - INFO - run_classifier -   input_ids: 101 3287 10527 25022 11890 2819 28472 2003 9092 15464 21723 2000 2775 6905 102 3008 2031 1996 2157 2000 2224 2037 2190 8689 1010 1999 1996 2422 1997 2966 6040 102
05/21/2020 13:29:01 - INFO - run_classifier -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
05/21/2020 13:29:01 - INFO - run_classifier -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
05/21/2020 13:29:01 - INFO - run_classifier -   label: 0 (id = 0)
05/21/2020 13:29:01 - INFO - run_classifier -   *** Example ***
05/21/2020 13:29:01 - INFO - run_classifier -  

In [16]:
model.train()
for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
#         cos_loss = model()
#         cos_loss = torch.nn.CosineEmbeddingLoss()
        
        print(loss)
        print(loss.item())
        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
#         if fp16 and loss_scale != 1.0:
#             # rescale loss for fp16 training
#             # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
#             loss = loss * loss_scale
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        loss.backward()
        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % gradient_accumulation_steps == 0:
#             if fp16 or optimize_on_cpu:
#                 if fp16 and loss_scale != 1.0:
#                     # scale down gradients for fp16 training
#                     for param in model.parameters():
#                         if param.grad is not None:
#                             param.grad.data = param.grad.data / loss_scale           
#                 is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
#                 if is_nan:
#                     logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
#                     loss_scale = loss_scale / 2
#                     model.zero_grad()
#                     continue 
#                 optimizer.step()
# #                 scheduler.step()
#                 copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
#             else:
#                 torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
#                 scheduler.step()
            model.zero_grad()
            global_step += 1

torch.save(model.state_dict(), output_dir + "dataExpantion.pth")

Iteration:   0%|                                                                               | 0/415 [00:00<?, ?it/s]

tensor(0.6337, device='cuda:0', grad_fn=<NllLossBackward>)
0.6337074041366577



Iteration:   0%|▏                                                                      | 1/415 [00:00<05:00,  1.38it/s]

tensor(0.5101, device='cuda:0', grad_fn=<NllLossBackward>)
0.510123074054718



Iteration:   0%|▎                                                                      | 2/415 [00:01<04:55,  1.40it/s]

...


Iteration: 100%|████████████████████████████████████████████████████████████████████▋| 413/415 [06:03<00:01,  1.13it/s]

tensor(0.5178, device='cuda:0', grad_fn=<NllLossBackward>)
0.5178437829017639



Iteration: 100%|████████████████████████████████████████████████████████████████████▊| 414/415 [06:04<00:00,  1.15it/s]

tensor(0.1020, device='cuda:0', grad_fn=<NllLossBackward>)
0.10197137296199799



Epoch: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [17:15<00:00, 345.26s/it]


In [12]:
import csv
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
def train_and_test(data_dir, bert_model="bert-base-uncased", task_name=None,
                   output_dir=None, max_seq_length=32, do_train=False, do_eval=False, do_lower_case=False,
                   train_batch_size=32, eval_batch_size=8, learning_rate=5e-5, num_train_epochs=3,
                   warmup_proportion=0.1,no_cuda=False, local_rank=-1, seed=42, gradient_accumulation_steps=1,
                   optimize_on_cpu=False, fp16=False, loss_scale=128, saved_model=""):


    # ## Required parameters
    # parser.add_argument("--data_dir",
    #                     default=None,
    #                     type=str,
    #                     required=True,
    #                     help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    # parser.add_argument("--bert_model", default=None, type=str, required=True,
    #                     help="Bert pre-trained model selected in the list: bert-base-uncased, "
    #                          "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.")
    # parser.add_argument("--task_name",
    #                     default=None,
    #                     type=str,
    #                     required=True,
    #                     help="The name of the task to train.")
    # parser.add_argument("--output_dir",
    #                     default=None,
    #                     type=str,
    #                     required=True,
    #                     help="The output directory where the model checkpoints will be written.")

    ## Other parameters
    # parser.add_argument("--max_seq_length",
    #                     default=128,
    #                     type=int,
    #                     help="The maximum total input sequence length after WordPiece tokenization. \n"
    #                          "Sequences longer than this will be truncated, and sequences shorter \n"
    #                          "than this will be padded.")
    # parser.add_argument("--do_train",
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to run training.")
    # parser.add_argument("--do_eval",
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to run eval on the dev set.")
    # parser.add_argument("--do_lower_case",
    #                     default=False,
    #                     action='store_true',
    #                     help="Set this flag if you are using an uncased model.")
    # parser.add_argument("--train_batch_size",
    #                     default=32,
    #                     type=int,
    #                     help="Total batch size for training.")
    # parser.add_argument("--eval_batch_size",
    #                     default=8,
    #                     type=int,
    #                     help="Total batch size for eval.")
    # parser.add_argument("--learning_rate",
    #                     default=5e-5,
    #                     type=float,
    #                     help="The initial learning rate for Adam.")
    # parser.add_argument("--num_train_epochs",
    #                     default=3.0,
    #                     type=float,
    #                     help="Total number of training epochs to perform.")
    # parser.add_argument("--warmup_proportion",
    #                     default=0.1,
    #                     type=float,
    #                     help="Proportion of training to perform linear learning rate warmup for. "
    #                          "E.g., 0.1 = 10%% of training.")
    # parser.add_argument("--no_cuda",
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether not to use CUDA when available")
    # parser.add_argument("--local_rank",
    #                     type=int,
    #                     default=-1,
    #                     help="local_rank for distributed training on gpus")
    # parser.add_argument('--seed',
    #                     type=int,
    #                     default=42,
    #                     help="random seed for initialization")
    # parser.add_argument('--gradient_accumulation_steps',
    #                     type=int,
    #                     default=1,
    #                     help="Number of updates steps to accumulate before performing a backward/update pass.")
    # parser.add_argument('--optimize_on_cpu',
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to perform optimization and keep the optimizer averages on CPU")
    # parser.add_argument('--fp16',
    #                     default=False,
    #                     action='store_true',
    #                     help="Whether to use 16-bit float precision instead of 32-bit")
    # parser.add_argument('--loss_scale',
    #                     type=float, default=128,
    #                     help='Loss scaling, positive power of 2 values can improve fp16 convergence.')

    # args = parser.parse_args()

    processors = {
#         "cola": ColaProcessor,
#         "mnli": MnliProcessor,
        "mrpc": MrpcProcessor,
    }

    if local_rank == -1 or no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        device = torch.device("cuda", local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
        if fp16:
            logger.info("16-bits training currently not supported in distributed training")
            fp16 = False # (see https://github.com/pytorch/pytorch/pull/13496)
    logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, bool(local_rank != -1))

    if gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            gradient_accumulation_steps))

    train_batch_size = int(train_batch_size / gradient_accumulation_steps)

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

    if not do_train and not do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")

    if do_train:
        if os.path.exists(output_dir) and os.listdir(output_dir):
            raise ValueError("Output directory ({}) already exists and is not emp1ty.".format(output_dir))
        os.makedirs(output_dir, exist_ok=True)

    task_name = task_name.lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name]()
    label_list = processor.get_labels()

    tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=do_lower_case)

    train_examples = None
    num_train_steps = None
    if do_train:
        train_examples = processor.get_train_examples(data_dir)
        num_train_steps = int(
            len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

    # Prepare model
    model = BertForSequenceClassification.from_pretrained(bert_model,
                cache_dir=PYTORCH_PRETRAINED_BERT_CACHE / 'distributed_{}'.format(local_rank), num_labels = 2)
    if fp16:
        model.half()
    model.to(device)
    if local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[local_rank],
                                                          output_device=local_rank)
    elif n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Prepare optimizer
    if fp16:
        param_optimizer = [(n, param.clone().detach().to('cpu').float().requires_grad_()) \
                            for n, param in model.named_parameters()]
    elif optimize_on_cpu:
        param_optimizer = [(n, param.clone().detach().to('cpu').requires_grad_()) \
                            for n, param in model.named_parameters()]
    else:
        param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
        ]
    t_total = num_train_steps
#     print(t_total)
    if local_rank != -1:
        t_total = t_total // torch.distributed.get_world_size()
    if do_train:
        optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)

    global_step = 0
    if do_train:
        train_features = convert_examples_to_features(
            train_examples, label_list, max_seq_length, tokenizer)
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(train_examples))
        logger.info("  Batch size = %d", train_batch_size)
        logger.info("  Num steps = %d", num_train_steps)
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        if local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

        model.train()
        for _ in trange(int(num_train_epochs), desc="Epoch"):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch
                loss = model(input_ids, segment_ids, input_mask, label_ids)
                if n_gpu > 1:
                    loss = loss.mean() # mean() to average on multi-gpu.
                if fp16 and loss_scale != 1.0:
                    # rescale loss for fp16 training
                    # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
                    loss = loss * loss_scale
                if gradient_accumulation_steps > 1:
                    loss = loss / gradient_accumulation_steps
                loss.backward()
                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                if (step + 1) % gradient_accumulation_steps == 0:
                    if fp16 or optimize_on_cpu:
                        if fp16 and loss_scale != 1.0:
                            # scale down gradients for fp16 training
                            for param in model.parameters():
                                if param.grad is not None:
                                    param.grad.data = param.grad.data / loss_scale
                        is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
                        if is_nan:
                            logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                            loss_scale = loss_scale / 2
                            model.zero_grad()
                            continue
                        optimizer.step()
                        copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
                    else:
                        optimizer.step()
                    model.zero_grad()
                    global_step += 1

        torch.save(model.state_dict(), output_dir + "output.pth")


    if do_eval and (local_rank == -1 or torch.distributed.get_rank() == 0):
        eval_examples = processor.get_test_examples(data_dir)
#         eval_examples = processor.get_dev_examples(data_dir)
        eval_features = convert_examples_to_features(
            eval_examples, label_list, max_seq_length, tokenizer)
        logger.info("***** Running evaluation *****")
        logger.info("  Num examples = %d", len(eval_examples))
        logger.info("  Batch size = %d", eval_batch_size)
        
        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
        eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        # Run prediction for full data
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

        model.load_state_dict(torch.load(saved_model))

        model.eval()
        # eval_loss, eval_accuracy = 0, 0

        eval_tp, eval_pred_c, eval_gold_c = 0, 0, 0
        eval_loss, eval_macro_p, eval_macro_r = 0, 0, 0

        raw_score = []

        nb_eval_steps, nb_eval_examples = 0, 0
        for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            label_ids = label_ids.to(device)

            with torch.no_grad():
                tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
                logits = model(input_ids, segment_ids, input_mask)
#             print(logits)
#             print(logits[0])
            logits = logits.detach().cpu().numpy()
            print(logits)
            label_ids = label_ids.to('cpu').numpy()

            # Micro F1 (aggregated tp, fp, fn counts across all examples)
            tmp_tp, tmp_pred_c, tmp_gold_c = tp_pcount_gcount(logits, label_ids)
            eval_tp += tmp_tp
            eval_pred_c += tmp_pred_c
            eval_gold_c += tmp_gold_c
            
            pred_label = np.argmax(logits, axis=1)
            raw_score += zip(logits, pred_label, label_ids)
            
            # Macro F1 (averaged P, R across mini batches)
            tmp_eval_p, tmp_eval_r, tmp_eval_f1 = p_r_f1(logits, label_ids)

            eval_macro_p += tmp_eval_p
            eval_macro_r += tmp_eval_r

            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_examples += input_ids.size(0)
            nb_eval_steps += 1


        # Micro F1 (aggregated tp, fp, fn counts across all examples)
        eval_micro_p = eval_tp / eval_pred_c
        eval_micro_r = eval_tp / eval_gold_c
        eval_micro_f1 = 2 * eval_micro_p * eval_micro_r / (eval_micro_p + eval_micro_r)

        # Macro F1 (averaged P, R across mini batches)
        eval_macro_p = eval_macro_p / nb_eval_steps
        eval_macro_r = eval_macro_r / nb_eval_steps
        eval_macro_f1 = 2 * eval_macro_p * eval_macro_r / (eval_macro_p + eval_macro_r)

        eval_loss = eval_loss / nb_eval_steps
        result = {'eval_loss': eval_loss,
                  'eval_micro_p': eval_micro_p,
                  'eval_micro_r': eval_micro_r,
                  'eval_micro_f1': eval_micro_f1,
                  'eval_macro_p': eval_macro_p,
                  'eval_macro_r': eval_macro_r,
                  'eval_macro_f1': eval_macro_f1,
#                   'global_step': global_step,
#                   'loss': tr_loss/nb_tr_steps
                  }

        output_eval_file = os.path.join(output_dir, "expansion_onlynewtest_eval_results.txt")
        output_raw_score = os.path.join(output_dir, "expansion_onlynewtest_raw_score.csv")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(result.keys()):
                logger.info("  %s = %s", key, str(result[key]))
                writer.write("%s = %s\n" % (key, str(result[key])))

        with open(output_raw_score, 'w') as fout:
            fields = ["undermine_score", "support_score","predict_label", "gold"]
            writer = csv.DictWriter(fout, fieldnames=fields)
            writer.writeheader()
            for score, pred, gold in raw_score:
                writer.writerow({
                    "undermine_score": str(score[0]),
                    "support_score": str(score[1]),
                    "predict_label": str(pred),
                    "gold": str(gold)
                })

In [ ]:
def experiments():
    data_dir = "D:/Jupyter/data/dataset/perspective_stances/"
#     data_dir = "/home/syg340/Dataset/"

    # data_dir_output = data_dir + "output2/"
    data_dir_output = "D:/Projects/Stance/Models/"
    train_and_test(data_dir=data_dir, do_train=True, do_eval=True, output_dir=data_dir_output,task_name="Mrpc")


In [13]:
def evaluation_with_pretrained():
    bert_model = "D:/Projects/Stance/Models/dataExpantion/dataExpantion.pth"
#     bert_model = "D:/Projects/Stance/Models/output.pth"
    data_dir = "D:/Jupyter/data/dataset/perspective_stances/"
#     data_dir = "D:/Projects/Stance/Dataset/BertForOppositeClassification/"
#     data_dir = "D:/Projects/Stance/Dataset/OnlyNew/"
    
    # data_dir_output = data_dir + "output2/"
    data_dir_output = "D:/Projects/Stance/Evaluation/bert_dummy_output/bert_experiment_local/dataExpansion/"
    train_and_test(data_dir=data_dir, do_train=False, do_eval=True, output_dir=data_dir_output,task_name="Mrpc",saved_model=bert_model)

In [14]:
if __name__ == "__main__":
#     experiments()
    evaluation_with_pretrained()

05/22/2020 15:52:53 - INFO - run_classifier -   device cuda n_gpu 1 distributed training False
05/22/2020 15:52:56 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\arsen\.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
05/22/2020 15:52:57 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\arsen\.pytorch_pretrained_bert\distributed_-1\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/22/2020 15:52:57 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file C:\Users\arsen\.pytorch_pretrained_bert\distributed_-1\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e

[[-0.22424129  0.10162512]
 [-0.63414747  0.9347689 ]
 [-0.9173301   1.3150159 ]
 [-0.53266555  0.7466488 ]
 [ 1.9179766  -1.9474862 ]
 [ 1.2523519  -1.2493237 ]
 [ 0.71249723 -0.6718124 ]
 [ 1.879803   -1.8280873 ]]
[[-2.5877125  2.8026059]
 [ 2.7025592 -2.5672638]
 [ 2.1051702 -2.1064832]
 [ 1.6868566 -1.5764197]
 [ 1.6450745 -1.5481588]
 [-2.2022514  2.5274713]
 [-1.4873993  1.8768169]
 [-2.6063151  2.821257 ]]
[[ 0.25668103 -0.43897614]
 [ 0.75658774 -0.71593857]
 [-2.616342    2.8069928 ]
 [-1.7179569   2.1325543 ]
 [-2.6064243   2.8129995 ]
 [ 1.949306   -1.7896918 ]
 [ 2.2538867  -2.1380858 ]
 [ 1.1394111  -1.0747354 ]]
[[ 1.8458898  -1.7002851 ]
 [ 1.697508   -1.6008514 ]
 [ 2.7798154  -2.697494  ]
 [ 2.7856112  -2.6971374 ]
 [ 2.337947   -2.3747084 ]
 [ 2.7477462  -2.6304083 ]
 [ 1.3358226  -1.2748033 ]
 [ 0.1178925  -0.25342506]]
[[-0.8597616   1.3621484 ]
 [ 0.47353226 -0.5871066 ]
 [-1.5139889   1.8995125 ]
 [-2.1154528   2.4660985 ]
 [-0.21039183  0.13271984]
 [-2.3731036 

[[-0.60222745  0.9852265 ]
 [ 0.25420064 -0.23562816]
 [-0.32781187  0.34887084]
 [-0.21107028  0.27455118]
 [ 0.09191899 -0.14667574]
 [-0.2366064   0.18756096]
 [ 0.5415966  -0.6262095 ]
 [ 2.250825   -2.2494624 ]]
[[-1.0079331   1.5067424 ]
 [-0.14248966 -0.00316283]
 [-1.2851335   1.7237002 ]
 [-0.24833256  0.37430167]
 [-0.1748576   0.04302571]
 [-0.14373262 -0.03817334]
 [ 1.0857991  -0.96756196]
 [-0.6421005   0.88342595]]
[[-2.6145794  2.806565 ]
 [ 1.8272303 -1.7364866]
 [ 2.7531998 -2.619768 ]
 [ 1.1176478 -1.1756757]
 [ 2.1840174 -2.1470451]
 [ 2.764947  -2.6989038]
 [ 1.4767504 -1.5511991]
 [ 1.8961945 -1.8098711]]
[[ 2.0500875  -1.9864239 ]
 [-1.2990136   1.8117799 ]
 [-2.6133158   2.8276422 ]
 [ 1.1458136  -1.2487797 ]
 [ 1.6914159  -1.7464465 ]
 [ 0.3497873  -0.48251343]
 [ 1.3234836  -1.3471137 ]
 [-0.5805137   0.95515144]]
[[ 1.1220909  -1.1586965 ]
 [-0.90717864  1.3464853 ]
 [ 0.371742   -0.4331396 ]
 [-0.50696963  0.67969996]
 [-1.312719    1.814805  ]
 [-0.39234832

[[-0.14965819  0.05382324]
 [-0.87400335  1.3750246 ]
 [ 2.5182989  -2.4162552 ]
 [ 1.3028628  -1.3371687 ]
 [-0.04772272  0.05302495]
 [ 1.3665863  -1.3110169 ]
 [ 2.6849656  -2.7020578 ]
 [ 2.22515    -2.1937313 ]]
[[-1.1807104   1.6901032 ]
 [-2.162942    2.4961488 ]
 [-1.8262702   2.2225966 ]
 [-0.58829916  1.0102009 ]
 [ 0.9295751  -0.8637059 ]
 [ 2.4840112  -2.5300193 ]
 [ 1.954222   -1.9622369 ]
 [-0.30450907  0.48471335]]
[[ 1.2874092  -1.1605753 ]
 [ 2.7801924  -2.6874602 ]
 [-1.802578    2.2141283 ]
 [ 0.16543847 -0.24055484]
 [ 2.4356556  -2.4238927 ]
 [ 2.7283225  -2.6198804 ]
 [ 2.578879   -2.4691622 ]
 [ 2.5662353  -2.4490695 ]]
[[ 1.106568   -1.2051528 ]
 [-0.45807934  0.63718176]
 [ 0.9152659  -0.96764493]
 [-1.4170383   1.8318436 ]
 [-1.1195778   1.5322638 ]
 [-1.3701783   1.8320696 ]
 [-2.4731765   2.7311342 ]
 [-1.1729233   1.6406184 ]]
[[-0.7784308   1.2400303 ]
 [-0.09647511  0.11587998]
 [-0.57743394  0.9352416 ]
 [-0.2140495   0.20060143]
 [-0.6298504   1.0774516

[[-1.1312293   1.6404485 ]
 [ 0.8567079  -0.9170586 ]
 [ 0.07685117 -0.24563713]
 [-0.22843231  0.23438402]
 [-0.9073647   1.4061092 ]
 [-0.12382492  0.21938434]
 [-0.5978534   1.0196587 ]
 [-0.22393447  0.18263304]]
[[ 0.35107672 -0.286372  ]
 [ 1.3521252  -1.2387241 ]
 [ 2.2401557  -2.1053507 ]
 [ 2.139825   -2.010629  ]
 [ 2.57338    -2.5620763 ]
 [ 2.7056675  -2.6488094 ]
 [-1.2333589   1.7337481 ]
 [-0.70710075  0.9535734 ]]
[[-0.10934324 -0.05614555]
 [-0.26942804  0.22817439]
 [-0.03211065 -0.15484227]
 [-0.30489072  0.24773268]
 [-1.8782746   2.2728436 ]
 [-1.0323515   1.5366849 ]
 [ 2.3097143  -2.1908457 ]
 [ 2.548666   -2.3887427 ]]
[[ 1.0979818 -1.1824043]
 [ 1.32798   -1.3739725]
 [-0.5380288  0.8027994]
 [ 2.779252  -2.6593125]
 [ 2.7556734 -2.658808 ]
 [ 2.6009276 -2.6089456]
 [ 2.3147154 -2.3304613]
 [ 2.62759   -2.5956478]]
[[ 0.83742    -0.88180614]
 [ 2.0764685  -2.0080018 ]
 [ 2.0615828  -2.023898  ]
 [ 2.4657936  -2.3177843 ]
 [ 0.3246013  -0.40054438]
 [-2.2187648 

...

[[ 0.5296017 -0.5001264]
 [-2.6281285  2.827555 ]
 [-2.4078865  2.6905446]
 [ 2.7239084 -2.5952554]
 [ 2.6615071 -2.6366153]
 [ 1.9345223 -1.9462645]
 [-1.4309267  1.8894862]
 [-1.1336027  1.6299686]]
[[-0.5962833   0.9744457 ]
 [ 2.5369449  -2.4046948 ]
 [-2.5202875   2.7683957 ]
 [ 2.2108607  -2.239489  ]
 [ 2.7018893  -2.674497  ]
 [ 2.7345126  -2.6658175 ]
 [-0.13234887 -0.06261697]
 [-0.14989616  0.14382562]]
[[-0.3060601   0.32722756]
 [-1.1644796   1.6173093 ]
 [-0.13638055  0.1217073 ]
 [-2.2745023   2.5745814 ]
 [-1.1065856   1.6092328 ]
 [-0.37203407  0.42911723]
 [-2.430645    2.6939213 ]
 [ 0.5278018  -0.4533093 ]]
[[-0.372267    0.4463883 ]
 [-0.23052037  0.27762598]
 [ 0.62179124 -0.6953621 ]
 [-0.16523452  0.02186853]
 [ 1.5101191  -1.55276   ]
 [ 1.224708   -1.2910718 ]
 [-0.30074638  0.30214736]
 [-0.4638757   0.5967005 ]]
[[-0.55646336  0.94698656]
 [ 1.9896837  -1.8679479 ]
 [ 1.6843851  -1.6512595 ]
 [-0.4579136   0.60786927]
 [ 0.09365416 -0.23283085]
 [ 2.02981   

[[ 2.6167386 -2.6370828]
 [ 0.4689712 -0.4440969]
 [ 1.6575078 -1.7450393]
 [-1.7100297  2.0871694]
 [-2.6131988  2.817035 ]
 [-1.2645391  1.7158691]
 [-2.5576074  2.7909403]
 [-2.2618613  2.5510771]]
[[-0.8058958   1.2942268 ]
 [ 2.4184427  -2.4046924 ]
 [ 0.14168821 -0.07419144]
 [-0.30093074  0.3862619 ]
 [ 2.6304862  -2.51123   ]
 [-2.6278102   2.8287842 ]
 [ 2.7164402  -2.6728058 ]
 [ 2.110867   -2.1544814 ]]
[[ 0.53589946 -0.65777314]
 [-0.20244737  0.1255797 ]
 [-0.5236147   0.7832836 ]
 [-0.11720203  0.2240141 ]
 [-0.26242426  0.3599154 ]
 [-0.08866268 -0.10992839]
 [-0.11989795 -0.11496025]
 [-0.20919456  0.13911197]]
[[ 2.6875625  -2.6360936 ]
 [ 1.270084   -1.2083094 ]
 [-0.3139098   0.41585603]
 [ 1.5637456  -1.6088883 ]
 [ 2.635386   -2.5922325 ]
 [ 2.7952502  -2.6779854 ]
 [ 1.8089933  -1.7969072 ]
 [ 2.7393413  -2.6946726 ]]
[[-0.6376534   1.0632695 ]
 [-0.80957806  1.3010832 ]
 [ 0.00310548 -0.1730939 ]
 [ 1.3151563  -1.3241783 ]
 [ 2.1714973  -2.1703985 ]
 [ 2.6530533 

[[ 1.2081372  -1.1935798 ]
 [ 2.6120625  -2.5645697 ]
 [-0.03339179 -0.17309412]
 [-1.0519526   1.5569062 ]
 [-1.0139943   1.5153133 ]
 [ 2.5185342  -2.5402167 ]
 [-0.07316402 -0.14315768]
 [-0.13144352 -0.07731066]]
[[ 1.2683872  -1.3555409 ]
 [-0.11615612 -0.08922743]
 [-1.4499948   1.9181865 ]
 [-0.9001533   1.4070094 ]
 [-0.11813142 -0.06752846]
 [-0.2733043   0.22310263]
 [ 0.6279304  -0.6450463 ]
 [-0.780439    1.2446653 ]]
[[-1.5536035   1.9888906 ]
 [-1.9476358   2.31417   ]
 [-1.2483817   1.7104644 ]
 [ 1.6425759  -1.6224072 ]
 [ 1.4962469  -1.5026985 ]
 [-0.19030555  0.11303044]
 [ 0.9703158  -1.0285873 ]
 [-0.09577045 -0.07031759]]
[[ 2.3747013  -2.4045672 ]
 [ 2.6082423  -2.5181277 ]
 [ 2.7457867  -2.634752  ]
 [ 1.590803   -1.5712126 ]
 [-2.6111531   2.830408  ]
 [-0.74348724  1.206251  ]
 [ 0.9974244  -0.95079356]
 [ 2.8226552  -2.736623  ]]
[[ 2.1658134  -2.1767576 ]
 [-2.3365452   2.6259408 ]
 [-2.6281676   2.8119948 ]
 [-2.6336641   2.8215492 ]
 [-1.1934706   1.5938903

05/22/2020 15:53:34 - INFO - run_classifier -   ***** Eval results *****
05/22/2020 15:53:34 - INFO - run_classifier -     eval_loss = 1.0616123614997803
05/22/2020 15:53:34 - INFO - run_classifier -     eval_macro_f1 = 0.6231499554089043
05/22/2020 15:53:34 - INFO - run_classifier -     eval_macro_p = 0.6235721820373717
05/22/2020 15:53:34 - INFO - run_classifier -     eval_macro_r = 0.6227283001808318
05/22/2020 15:53:34 - INFO - run_classifier -     eval_micro_f1 = 0.6041228439209086
05/22/2020 15:53:34 - INFO - run_classifier -     eval_micro_p = 0.6095076400679117
05/22/2020 15:53:34 - INFO - run_classifier -     eval_micro_r = 0.5988323603002502


[[ 2.729869   -2.6967318 ]
 [-0.47437024  0.7146379 ]
 [-0.00900108 -0.19315624]
 [ 1.2588012  -1.2651626 ]
 [-2.0932956   2.4212952 ]
 [-2.586116    2.8185408 ]
 [-0.56801933  0.9624652 ]]
